<a href="https://colab.research.google.com/github/ArifAygun/Webscraping/blob/main/AA_Largest_mining_companies_by_market_cap_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Web_Scraping/Companiesmarketcap/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Web_Scraping/Companiesmarketcap


In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [11]:
base_site1 = f"https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap"

response = requests.get(base_site1)
html = response.content
soup = BeautifulSoup(html, "html.parser")

rowCount = int(soup.find('span', {'class':'companies-count'}).text)
pageCount = int(np.ceil(rowCount/100))
pageCount

3

In [12]:
def pageCountFound():
    base_site1 = f"https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap"
    response = requests.get(base_site1)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    rowCount = int(soup.find('span', {'class':'companies-count'}).text)
    pageCount = int(np.ceil(rowCount/100))
    return pageCount

def finalPulling():
    df_out = pd.DataFrame()
    for pageNo in range(1, pageCountFound()+1):
        base_site = f"https://companiesmarketcap.com/mining/largest-mining-companies-by-market-cap/?page={pageNo}"
        response = requests.get(base_site)
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        tbody = soup.find('tbody')

        #alternative lines for ranks is below "if block"
        #ranks_list = tbody.find_all('td', {'class':'rank-td td-right'})
        #ranks = [i.text for i in ranks_list]

        name_list = tbody.find_all('td', {'class':'name-td'})
        names = [i.select('div', {'class':'company-name'})[2].text.strip() for i in name_list]

        name_link_list = tbody.find_all('div', {'class':'name-div'})
        name_links = [i.find("a").get("href") for i in name_link_list]

        cap_links = tbody.find_all('td', {'class':'td-right'})
        caps_links_list = [i.text for i in cap_links]

        if pageNo < pageCount:
            caps = [caps_links_list[i] for i in list(range(1,pageCount*100,3))]
            prices = [caps_links_list[i] for i in list(range(2,pageCount*100,3))]
            ranks = [caps_links_list[i] for i in list(range(0,pageCount*100,3))]
        else:
            caps = [caps_links_list[i] for i in list(range(1,(rowCount%((pageCount-1)*100)*3),3))]
            prices = [caps_links_list[i] for i in list(range(2,(rowCount%((pageCount-1)*100)*3),3))]
            ranks = [caps_links_list[i] for i in list(range(0,(rowCount%((pageCount-1)*100)*3),3))]

        percent_links = tbody.find_all('span', {'class':'percentage-red'})
        percents = [i.text for i in percent_links]
        country_links = tbody.find_all('span', {'class':'responsive-hidden'})
        countries = [i.text for i in country_links]

        df_in = pd.DataFrame(zip(ranks, names, name_links, caps, prices, percents, countries), columns =["ranks","names", "name_links", "caps", "prices", "percents", "countries"])
        df_out = pd.concat([df_out, df_in], axis = 0)

    df_out.to_csv("df_out.csv")
    return df_out

finalPulling()

,ranks,names,name_links,caps,prices,percents,countries
0,1,BHP Group,/bhp-group/marketcap/,$146.36 B,$57.84,0.70%,Australia
1,2,Rio Tinto,/rio-tinto/marketcap/,$106.63 B,$65.41,1.46%,UK
2,3,China Shenhua Energy,/china-shenhua-energy/marketcap/,$82.07 B,$4.33,3.41%,China
3,4,Glencore,/glencore/marketcap/,$70.42 B,$5.70,1.72%,Switzerland
4,5,Vale,/vale/marketcap/,$61.55 B,$14.12,1.13%,Brazil
...,...,...,...,...,...,...,...
2,203,Almaden Minerals,/almaden-minerals/marketcap/,$17.85 M,$0.13,11.11%,Canada
3,204,Aurora Energy Metals,/aurora-energy-metals/marketcap/,$11.51 M,$0.06000,2.17%,Australia
4,205,Austin Gold,/austin-gold/marketcap/,$10.7 M,$0.81,12.50%,USA
5,206,GTI Energy,/gti-energy/marketcap/,$10.52 M,$0.01000,1.53%,Australia
